In [13]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
import pandas as pd
from selenium.webdriver import ActionChains
import urllib.request as urllib
from bs4 import BeautifulSoup

In [14]:
# load website urls and create a list of urls
url_nba_odds = 'https://www.actionnetwork.com/nba/odds'
url_nfl_odds = 'https://www.actionnetwork.com/nfl/odds'
url_ncaab_odds = 'https://www.actionnetwork.com/ncaab/odds'
url_ncaaf_odds = 'https://www.actionnetwork.com/ncaaf/odds'
list_url_odds = [url_nba_odds, url_nfl_odds,url_ncaab_odds,url_ncaaf_odds]

In [15]:
# use selenium to create list of soup tables
driver = webdriver.Chrome(executable_path='/Users/connerlusk/Downloads/chromedriver')
list_soup = []
for odds in list_url_odds:
    driver.get(odds)
    moneyline = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[3]/div[2]/div[1]/label[3]/input")
    spread = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[3]/div[2]/div[1]/label[1]/input")
    action = ActionChains(driver)
    action.click(moneyline)
    action.click(spread)
    action.perform()
    sleep(4)
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    list_soup.append(soup)

<ipython-input-15-275762b50ad7>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/connerlusk/Downloads/chromedriver')
<ipython-input-15-275762b50ad7>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  moneyline = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[3]/div[2]/div[1]/label[3]/input")
<ipython-input-15-275762b50ad7>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  spread = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[3]/div[2]/div[1]/label[1]/input")


In [16]:
# scrapes the soup table
def clean_odds(soup):
  betting_title = soup.find(class_ = "custom-1jexfpi e1vl13ep0" )
  betting_odds_title = betting_title.h1.text
  the_sport_name = betting_odds_title.split()[0]

  odds_list = []

  soup_cu = soup.find_all('div', class_='custom-1u3dgby e4ukmj14')

  # Finds all tr tag in soup_table
  soup_table = soup_cu[0].find_all('tr')
  
  # list of books on website 
  book_list = ['Draft Kings', 'Bet MGM', 'Fanduel', 'PointsBet', 'Sugar House', 'Bet365', 'Unibet', 'Caesar' + "'s " + 'Sportsbook']

  ind_ = 1

  for game_ in soup_table[1:]:
    # create dicts for odds of each team
    team1_odds = dict()
    team2_odds = dict()
    
    # grabs div tag with specified class
    team_names = game_.find_all('div', class_='game-info__team')
    
    # grabs team names from tag as text
    name1 = team_names[0].find(class_='game-info__team--desktop').text
    name2 = team_names[1].find(class_='game-info__team--desktop').text
    
    # sets team names in dicts
    team1_odds['team'] = name1
    team2_odds['team'] = name2

    # access tag for game odds 
    game_layer1 = game_.find_all('td', class_='best-odds__last-type-cell')
      
    indx = 0

    # loops through book odds divided by specified classes
    for bookie in game_layer1:
      odds_team1 = bookie.find_all('span', 
                                     class_='highlight-text__children')[0].text
      odds_team2 = bookie.find_all('span', 
                                     class_='highlight-text__children')[-2].text
      
      # assigns name to dict entry
      book = book_list[indx]
      team1_odds[book] = odds_team1
      team2_odds[book] = odds_team2
      indx += 1

    ind_+=1

    # appends odds info into list
    odds_list.append(team1_odds)
    odds_list.append(team2_odds)
  
  # instantiates DataFrame of NBA odds
  df_odds = pd.DataFrame(odds_list)
  df_odds.columns.name = the_sport_name
  #return df_nba_odds
  return df_odds

In [17]:
# get all pandas df
odds_final = []
for sport in list_soup:
    odds_final.append(clean_odds(sport).replace(['N/A'],-100000000))


In [18]:
# find arbitrgae oppertunities
def find_arbitrage(bet_size, data_frame):
    bookies = data_frame.columns[1:]
    clean_data(data_frame, bookies)
    sport = data_frame.columns.name
    strings = []
    find_odds = []
    indx = 0
    while indx < len(data_frame):
        max_a = data_frame[bookies].iloc[indx].max()
        max_b = data_frame[bookies].iloc[indx + 1].max()
        if is_arbitrage(max_a, max_b):
            indx_sitea = (data_frame.iloc[indx] == max_a).argmax()
            team_a = [data_frame.iloc[indx][0],bookies[indx_sitea -1]]
            indx_siteb = (data_frame.iloc[indx + 1] == max_b).argmax()
            team_b = [data_frame.iloc[indx+1][0],bookies[indx_siteb -1]]
            strings.extend(calculate_arbitrage(sport,bet_size,max_a,max_b,team_a,team_b))
        indx += 2
    return strings

In [19]:
# calcuylate arbitrage oppertunities
def calculate_arbitrage(sport_name, bet_size, max_a, max_b,team_a,team_b):
    strings = []
    implied_prob_a = find_implied_prob(max_a)
    implied_prob_b = find_implied_prob(max_b)
    implied_prob_tot = implied_prob_a + implied_prob_b
    win_percent = round(100 - implied_prob_tot,2)
    a_bet = round((bet_size*implied_prob_a)/implied_prob_tot,2)
    b_bet = round((bet_size*implied_prob_b)/implied_prob_tot,2)
    a_biased_bet_b = round(bet_size / to_decimal(max_b),2)
    a_biased_bet_a = round(bet_size - a_biased_bet_b,2)
    win_percent_a_biased = (a_biased_bet_a * to_decimal(max_a)) - bet_size
    b_biased_bet_a = round(bet_size / to_decimal(max_a),2)
    b_biased_bet_b = round(bet_size - b_biased_bet_a,2)
    win_percent_b_biased = (b_biased_bet_b * to_decimal(max_b)) - bet_size
    if win_percent_a_biased > win_percent_b_biased:
        strings.append(sport_name +" weighted arbitrage oppertunity | Max win percent: " +str(round(win_percent_a_biased,2))+"% | " + team_a[0] + " " + 
                       team_a[1] + " $" + str(a_biased_bet_a) +" | "+team_b[0] + " " + team_b[1] + " $" + str(a_biased_bet_b))
    else:
        strings.append(sport_name +" weighted arbitrage oppertunity | Max win percent: " +str(round(win_percent_b_biased,2))+"% | " + team_a[0] + " " + 
                       team_a[1] + " $" + str(b_biased_bet_a) +" | "+team_b[0] + " " + team_b[1] + " $" + str(b_biased_bet_b))
        
    strings.append(sport_name + " arbitrage oppertunity | Max win percent: " +str(win_percent)+"% | " + team_a[0] + " " + team_a[1] + " $" + str(a_bet) +" | "+team_b[0] + " " + team_b[1] + " $" + str(b_bet))
    return strings  

In [20]:
# sees if there is a arbitrage oppertunity between two teams
def is_arbitrage(max1, max2):
    implied_prob1 = find_implied_prob(max1)
    implied_prob2 = find_implied_prob(max2)
    return implied_prob1 + implied_prob2 < 100

In [21]:
# finds the implied probability of a team
def find_implied_prob(max_num):
    if max_num > 0:
        return 100 / ((max_num + 100)) * 100
    else:
        return (-max_num/(-max_num + 100)) * 100

In [22]:
# changes american odds to decimal odds
def to_decimal(american_odds):
    if american_odds > 0:
        return (american_odds/100)+1
    else:
        return 1 + (100/-american_odds)

In [23]:
# cleans the data
def clean_data(data_frame, bookies):
    for bookie in bookies:
        sport[bookie] = sport[bookie].astype(int)
    

In [24]:
# calculates arbitrage oppertunites for all sports
final_arbitage = []
for sport in odds_final:
    final_arbitage.extend(find_arbitrage(100, sport))
final_arbitage


['NFL weighted arbitrage oppertunity | Max win percent: 1.77% | Giants Sugar House $35.71 | Dolphins Bet365 $64.29',
 'NFL arbitrage oppertunity | Max win percent: 0.63% | Giants Sugar House $35.31 | Dolphins Bet365 $64.69',
 'NFL weighted arbitrage oppertunity | Max win percent: 10.87% | 49ers PointsBet $56.52 | Seahawks Sugar House $43.48',
 'NFL arbitrage oppertunity | Max win percent: 4.26% | 49ers PointsBet $59.04 | Seahawks Sugar House $40.96',
 'NCAAB weighted arbitrage oppertunity | Max win percent: 18.17% | Furman Draft Kings $81.82 | High Point Fanduel $18.18',
 'NCAAB arbitrage oppertunity | Max win percent: 2.8% | Furman Draft Kings $84.17 | High Point Fanduel $15.83',
 'NCAAB weighted arbitrage oppertunity | Max win percent: 2.26% | Prairie View Fanduel $7.41 | SD State PointsBet $92.59',
 'NCAAB arbitrage oppertunity | Max win percent: 0.16% | Prairie View Fanduel $7.26 | SD State PointsBet $92.74',
 'NCAAB weighted arbitrage oppertunity | Max win percent: 28.33% | NW Sta